In [7]:
# Pandas is used for data manipulation
import pandas as pd
import numpy as np

# Read in data 
z_data = pd.read_csv('fse17_act_deact_zscore_groups.csv',sep=';',decimal='.')

# Remove irrelevant columns
z_data.drop(list(z_data.filter(regex = '_deact')),axis = 1, inplace = True)
z_data.drop(list(z_data.filter(regex = 'aggr')),axis = 1, inplace = True)


#first, drop everything non-numeric ===========================================================
z_data = z_data.drop('scan', axis = 1)
z_data = z_data.drop('snippet', axis = 1)
z_data = z_data.drop('response', axis = 1)

#=== now create the different groups and compute the groupwise mean ============================
grouped = z_data.groupby(['proband', 'trial','task'])
groupedAgg = grouped.aggregate(np.mean)
labels = groupedAgg.index.get_level_values(level='task')
#=== end aggregation ===========================================================================

#=== creating training and validation set ======================================================
# Saving feature names for later use
feature_list = list(groupedAgg.columns)

# Convert to numpy array
features = np.array(groupedAgg)
labels = np.array(labels)


numLabels = 2 # num of different labels (comprehension and rest)
sizeTrainSet = 11 # num of participants used for training
sizeTestSet = 3 # num participants used for testing
 
# get the number of trials per participant
numTrials = len(groupedAgg.index.get_level_values(level='trial').unique())
numParticipants = sizeTrainSet + sizeTestSet


#get the rows excluding the last indexed (i.e., row with the index 587 is the last to include)
training_features = features[0:numTrials*sizeTrainSet]
training_features = np.array(training_features)
testing_features = features[numTrials*sizeTrainSet:numTrials*numParticipants]
testing_features = np.array(testing_features)

training_target = labels[0:numTrials*sizeTrainSet]
training_target = np.array(training_target)
testing_target = labels[numTrials*sizeTrainSet:numTrials*numParticipants]
testing_target = np.array(testing_target)
#=== end creating training and validation set ===================================================


C:\Users\author\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (2204,2205,2206,2207,2208,2209,2210,2211,2212,2213,2214,2215,2216,2217,2218,2219,2220,2221,2222,2223) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
print(training_features.shape)
print(testing_features.shape)
print(training_target.shape)
print(testing_target.shape)

(638, 390)
(174, 390)
(638,)
(174,)


In [2]:
from tpot import TPOTClassifier
fileNameTPot = 'Act/tpot_mnist_pipeline_FineAverageParticipantSplit_Act'

tpot = TPOTClassifier(generations=5, population_size=20, verbosity=2, n_jobs=20)
tpot.fit(training_features, training_target)
print(tpot.score(testing_features, testing_target))
tpot.export(fileNameTPot + '.py')

C:\Users\author\Anaconda3\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)
C:\Users\author\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\author\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\author\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\author\Anaconda3\lib\importlib\_bootstrap_external.py:426: ImportWarning: Not importing directory C:\Users\author\Anaconda3\lib\site-packages\mpl_toolkits: missing __init__
  _warnin

Generation 1 - Current best internal CV score: 0.8465859236708784


Generation 2 - Current best internal CV score: 0.8528726423731918


Generation 3 - Current best internal CV score: 0.8558392274003541


Generation 4 - Current best internal CV score: 0.8558392274003541


Generation 5 - Current best internal CV score: 0.8558392274003541



Best pipeline: LinearSVC(LinearSVC(MaxAbsScaler(input_matrix), C=0.001, dual=False, loss=squared_hinge, penalty=l2, tol=1e-05), C=0.1, dual=False, loss=squared_hinge, penalty=l1, tol=0.1)
0.8563218390804598


True

In [3]:
with open(fileNameTPot + '.py') as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
# content = [x.strip() for x in content] 

#      or 'exported_pipeline = ' not in line
cleanedContent = []
for line in content:
    if 'tpot_data' not in line and 'training_target, testing_target' not in line:
        cleanedContent.append(line)

fileForLearning = fileNameTPot + '_cleaned.py'
with open(fileForLearning, 'w') as filehandle:  
    for line in cleanedContent:
        filehandle.write('%s\n' % line)

In [4]:
%run -i $fileForLearning

In [6]:
num_matches = 0;
for a, b in zip(testing_target, results):
    if a == b:
        num_matches = num_matches + 1
print('Number of matches:',num_matches,'(of',testing_target.size,')')

accuracy = num_matches/testing_target.size*100
print('Accuary: ',accuracy)


Number of matches: 150 (of 174 )
Accuary:  86.20689655172413


In [7]:
with open('listOfAccuracies.txt','a+') as f:
    f.write('%s\n' % 'Fine_Act' + str(accuracy) + '\n')
